# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [27]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [28]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\gabri\Downloads


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [29]:
filename = 'yakult-final.xlsx'

In [30]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,@colchoesueli a médica passou lactobacilos p...,1.0
1,alguém compra yakult pra mim por deus,0.0
2,"amanhã vou fazer aquele drink de morango, ...",0.0
3,tomando um yakult no meio da rua 💪,1.0
4,48 horas e estou sobrevivendo com;\\\\nmei...,1.0


In [31]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificador
0,"@otavio_amaral18 é uma delícia juro, desce...",1
1,"o cérebro é muito impressionante, ontem eu...",1
2,@fenicolli02 quem q pede gole do yakult???...,0
3,@thamaramaia5 @adrianommartins @taaayrd @t...,0
4,sdds das mamis juntas brindando yakult @fe...,0


In [64]:
Irrelevantes=train.loc[train['Classificação']==0,'Treinamento']
Irrelevantes

1                alguém compra yakult pra mim por deus  
2          amanhã vou fazer aquele drink de morango, ...
5          @madrugatina sei que vc não bebe, então va...
6          meu lindo :( quero ver ele fazendo batida ...
7          nadei igual um desgraçado, agora comi bata...
                             ...                        
482        talvez eu faça aquela bebida de morango co...
486        meu maior sonho atualmente é um yakult veg...
492        @bcamoezi esses dias comprei couve, alface...
495        ontem eu comi...\\\\npão na chapa\\\\nstak...
497        ontem eu cheguei de madrugada em casa e fi...
Name: Treinamento, Length: 285, dtype: object

In [65]:
Relevantes=train.loc[train['Classificação']==1,'Treinamento']
Relevantes

0        @colchoesueli a médica passou lactobacilos p...
3                    tomando um yakult no meio da rua 💪 
4          48 horas e estou sobrevivendo com;\\\\nmei...
14         minha sogra fala mais que a mulher do yaku...
15         última coisa q comeu — tomei um yakult de ...
                             ...                        
490        a garrafinha de yakult é mt pequena, mano,...
491          @caarolmario bebida de yakult com corote   
493        acabei de ingerir isso aqui, me disseram q...
494        bernardo aprendeu a tomar yakult de canudi...
496      bolo: red velvet ou chocolate\\\\ndoce: chee...
Name: Treinamento, Length: 213, dtype: object

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto que foi escolhido para ser analizado foi o yakult, a famosa bebida fermentada japonesa, o grupo levou em consideração a emissão de opinião na hora de montar o seu classificador, se emitisse alguma opinião seria considerado relevante, caso contrário 

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [110]:
import re 


def cleanup(texto):
    pontuacao = "[!-.:?+_] https"
    pattern = re.compile(pontuacao)
    texto_novo = re.sub(pattern, '', texto)
    return texto_novo

In [111]:
texto_relevante=' '.join(Relevantes)

In [112]:
texto_irrelevante=' '.join(Irrelevantes)

In [113]:
texto_limpo_relevante=cleanup(texto_relevante).lower()
texto_limpo_irrelevante=cleanup(texto_relevante).lower()

In [114]:
palavras_relevantes = texto_limpo_relevante.split()
todas_palavras_relevantes = pd.Series(palavras_relevantes)

In [115]:
palavras_irrelevantes = texto_limpo_irrelevante.split()
todas_palavras_irrelevantes = pd.Series(palavras_irrelevantes)

In [116]:
serie_irrelevantes_relativa = todas_palavras_irrelevantes.value_counts(True)

In [117]:
serie_relevantes_relativa = todas_palavras_relevantes.value_counts(True)

In [118]:
serie_relevantes = pd.Series(todas_palavras_relevantes)
tabela_relevantes = serie_relevantes.value_counts()

In [119]:
serie_irrelevantes = pd.Series(todas_palavras_irrelevantes)
tabela_irrelevantes = serie_irrelevantes.value_counts()

In [120]:
todas = texto_limpo_relevante + texto_limpo_irrelevante

In [121]:
irrelevantes_palavras=tabela_irrelevantes.sum()
relevantes_palavras=tabela_relevantes.sum()
todas=irrelevantes_palavras+relevantes_palavras
probI=irrelevantes_palavras/todas
probR=relevantes_palavras/todas

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**